In [ ]:
!pip install datasets

In [4]:
from datasets import load_dataset


In [112]:
import pandas as pd

In [18]:
import random
random.seed(1)  # For consistency between the runs


In [128]:
class GenerateDataset():
  def __init__(self, dataset,length,is_train=True,min_length=4):
    self.dataset = load_dataset(dataset) # Can take a lot of time depending upon dataset
    if is_train:
      self.dataset=self.dataset["train"]
    self.length=length
    self.min_length=min_length
    self.dataset=self.get_dataset()
    self.__dataset=self.preprocess(self.dataset)  # For debugging purposes only
    self.dataset=self.gen_incomplete(self.__dataset)

  @property
  def complete_dataset(self):
    return self.__dataset

  def get_dataset(self,param=None):
    if param==None:
      premlim_li=random.choices(self.dataset,k=self.length)
    else:
      premlim_li=random.choices(self.dataset,k=param)
    return premlim_li

  def preprocess(self,li):
    buffer=[]
    count=0
    max_iter=100
    while True:
      max_iter-=1
      if max_iter==0:
        break
      for i in li:
        if len(i["text"].strip().split())<=self.min_length:
          count+=1
        else:
          buffer.append(i["text"])
      if count==0 or len(buffer)==self.length:
        break
      else:
        li=self.get_dataset(count)
        count=0
    return buffer

  def gen_incomplete(self,pre_li):
    buffer=[]
    for i in pre_li:
      arr=i.strip().split()
      buffer.append(" ".join(arr[:(len(arr)//2)]))

    return buffer



  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    return self.dataset[idx]

  def toCSV(self):
    df=pd.DataFrame([i for i in self.dataset],columns=["x_i"])
    df.to_csv("dataset.csv")


In [129]:
data=GenerateDataset("bookcorpus",100)

In [130]:
data.toCSV()

In [131]:
data[50]

'it was the only thing'

In [132]:
data.complete_dataset[50]  # For debugging only, just to see what full sentence looks like

'it was the only thing that was holding him back .'